Adapted from Chapter 8 (Reshaping and Pivoting) of Book Pandas in Action, O'Reilly

# 1 - Pivoting

In [1]:
import pandas as pd

In [2]:
sales = pd.read_csv("sales_by_employee.csv",
                    parse_dates = ["Date"])

sales.tail()

Date   Name           Customer  Revenue  Expenses
21 2020-01-01  Creed        Money Corp.     4430       548
22 2020-01-02  Creed  Average Paper Co.     8026      1906
23 2020-01-02  Creed  Average Paper Co.     5188      1768
24 2020-01-04  Creed         PaperMaven     3144      1314
25 2020-01-05  Creed        Money Corp.      938      1053

## The pivot_table Method

A pivot table __aggregates column’s values and groups the results by using other columns’ values__. 

The word aggregate describes a summary computation that involves multiple values. 

Example aggregations include average, sum, median, and count. 

A pivot table in pandas is similar to the Pivot Table feature in Microsoft Excel.

Steps to create a pivot table:

* Select the column(s) whose values we want to aggregate.

* Choose the aggregation operation to apply to the column(s).

* Select the column(s) whose values will group the aggregated data into categories.

* Determine whether to place the groups on the row axis, the column axis, or both axes.

### Example 1

The __index__ parameter accepts the column whose values will make up the pivot table’s index labels

Pandas applies its __default aggregation operation, an average, to all numeric columns__

The method returns a regular DataFrame object

In [3]:
sales.pivot_table(index = "Date")

NumExpr defaulting to 4 threads.


Expenses      Revenue
Date                                
2020-01-01   637.500000  4293.500000
2020-01-02  1244.400000  7303.000000
2020-01-03  1313.666667  4865.833333
2020-01-04  1450.600000  3948.000000
2020-01-05  1196.250000  4834.750000

### Example 2

We declare the aggregation function with the __aggfunc__ parameter; 
its default argument is "mean". 

The following code produces the same result as the preceding code:

In [4]:
sales.pivot_table(index = "Date", aggfunc = "mean")

Expenses      Revenue
Date                                
2020-01-01   637.500000  4293.500000
2020-01-02  1244.400000  7303.000000
2020-01-03  1313.666667  4865.833333
2020-01-04  1450.600000  3948.000000
2020-01-05  1196.250000  4834.750000

To get sum of each date’s revenue organized by date we can use:

In [5]:
sales.pivot_table(index = "Date", aggfunc = "sum")

Expenses  Revenue
Date                         
2020-01-01      3825    25761
2020-01-02      6222    36515
2020-01-03      7882    29195
2020-01-04      7253    19740
2020-01-05      4785    19339

### Example 3

The __values__ parameter accepts the DataFrame column(s) that pandas will aggregate. 

To aggregate only one column’s values, we can pass the parameter a string with the column name:

In [6]:
sales.pivot_table(
    index = "Date", values = "Revenue", aggfunc = "sum"
)

Revenue
Date               
2020-01-01    25761
2020-01-02    36515
2020-01-03    29195
2020-01-04    19740
2020-01-05    19339

### Example 4

We have a sum of revenue grouped by date. 

We are insterested in communicating how much each salesman contributed to the daily total. 

One presentational view that seems to be optimal is placing each salesman’s name in a separate column. 
In other words, we’d like to use the Name column’s unique values as the column headers in the pivot table. 

Let’s add a __columns__ parameter to the method invocation and pass it an argument of "Name":


In [7]:
sales.pivot_table(
    index = "Date",
    columns = "Name",
    values = "Revenue",
    aggfunc = "sum"
)

Name          Creed   Dwight     Jim  Michael   Oscar
Date                                                 
2020-01-01   4430.0   2639.0  1864.0   7172.0  9656.0
2020-01-02  13214.0      NaN  8278.0   6362.0  8661.0
2020-01-03      NaN  11912.0  4226.0   5982.0  7075.0
2020-01-04   3144.0      NaN  6155.0   7917.0  2524.0
2020-01-05    938.0   7771.0     NaN   7837.0  2793.0

Our resulting table has NaNs.

We can use the __fill_value__ parameter to replace all pivot table NaNs with a fixed value. 

Let’s fill in the data gaps with zeroes:

In [8]:
sales.pivot_table(
    index = "Date", 
    columns = "Name", 
    values = "Revenue", 
    aggfunc = "sum",
    fill_value = 0
)

Name        Creed  Dwight   Jim  Michael  Oscar
Date                                           
2020-01-01   4430    2639  1864     7172   9656
2020-01-02  13214       0  8278     6362   8661
2020-01-03      0   11912  4226     5982   7075
2020-01-04   3144       0  6155     7917   2524
2020-01-05    938    7771     0     7837   2793

We may also want to see the revenue subtotals for each combination of date and salesman. 

We can pass an argument of True to the __margins__ parameter to add totals for each row and column:

In [9]:
sales.pivot_table(
    index = "Date",
    columns = "Name",
    values = "Revenue",
    aggfunc = "sum",
    fill_value = 0,
    margins = True
)

Name                 Creed  Dwight    Jim  Michael  Oscar     All
Date                                                             
2020-01-01 00:00:00   4430    2639   1864     7172   9656   25761
2020-01-02 00:00:00  13214       0   8278     6362   8661   36515
2020-01-03 00:00:00      0   11912   4226     5982   7075   29195
2020-01-04 00:00:00   3144       0   6155     7917   2524   19740
2020-01-05 00:00:00    938    7771      0     7837   2793   19339
All                  21726   22322  20523    35270  30709  130550

We can use the margins_name parameter to customize the subtotal labels. 

The next example changes the labels from "All" to "Total":

In [10]:
sales.pivot_table(
    index = "Date", 
    columns = "Name", 
    values = "Revenue", 
    aggfunc = "sum",
    fill_value = 0,
    margins = True,
    margins_name = "Total"
)

Name                 Creed  Dwight    Jim  Michael  Oscar   Total
Date                                                             
2020-01-01 00:00:00   4430    2639   1864     7172   9656   25761
2020-01-02 00:00:00  13214       0   8278     6362   8661   36515
2020-01-03 00:00:00      0   11912   4226     5982   7075   29195
2020-01-04 00:00:00   3144       0   6155     7917   2524   19740
2020-01-05 00:00:00    938    7771      0     7837   2793   19339
Total                21726   22322  20523    35270  30709  130550

### Additional Options for Pivot Tables

A pivot table __supports a variety of aggregation operations__. 

Suppose that we’re interested in the number of business deals closed per day. 

We can pass aggfunc an argument of "count" to count the number of sales rows for each combination of date and employee:

In [11]:
sales.pivot_table(
    index = "Date", 
    columns = "Name",
    values = "Revenue",
    aggfunc = "count"
)

Name        Creed  Dwight  Jim  Michael  Oscar
Date                                          
2020-01-01    1.0     1.0  1.0      1.0    2.0
2020-01-02    2.0     NaN  1.0      1.0    1.0
2020-01-03    NaN     3.0  1.0      1.0    1.0
2020-01-04    1.0     NaN  2.0      1.0    1.0
2020-01-05    1.0     1.0  NaN      1.0    1.0

We can also pass a __list of aggregation functions__ to the pivot_table function’s aggfunc parameter. 

The pivot table will create a MultiIndex on the column axis and store the aggregations in its outermost level.

In [12]:
sales.pivot_table(
    index = "Date",
    columns = "Name",
    values = "Revenue",
    aggfunc = ["sum", "count"],
    fill_value = 0
)

sum                            count                         
Name        Creed Dwight   Jim Michael Oscar Creed Dwight Jim Michael Oscar
Date                                                                       
2020-01-01   4430   2639  1864    7172  9656     1      1   1       1     2
2020-01-02  13214      0  8278    6362  8661     2      0   1       1     1
2020-01-03      0  11912  4226    5982  7075     0      3   1       1     1
2020-01-04   3144      0  6155    7917  2524     1      0   2       1     1
2020-01-05    938   7771     0    7837  2793     1      1   0       1     1

We can apply __different aggregations to different columns__ by passing a __dictionary__ to the aggfunc parameter. 

Use the dictionary’s keys to identify DataFrame columns and the values to set the aggregation.

In [13]:
sales.pivot_table(
    index = "Date",
    columns = "Name", 
    values = ["Revenue", "Expenses"],
    fill_value = 0,
    aggfunc = { "Revenue": "min", "Expenses": "min" }
)

Expenses                            Revenue                       \
Name          Creed Dwight   Jim Michael Oscar   Creed Dwight   Jim Michael   
Date                                                                          
2020-01-01      548    368  1305     412   531    4430   2639  1864    7172   
2020-01-02     1768      0   462     685  1401    5188      0  8278    6362   
2020-01-03        0    758  1923    1772   906       0   2703  4226    5982   
2020-01-04     1314      0   426    1857  1767    3144      0  2287    7917   
2020-01-05     1053   1475     0    1633   624     938   7771     0    7837   

                  
Name       Oscar  
Date              
2020-01-01  4406  
2020-01-02  8661  
2020-01-03  7075  
2020-01-04  2524  
2020-01-05  2793

We can also stack multiple groupings on a single axis by __passing the index parameter a list of columns__. 

The next example aggregates the sum of expenses by salesman and date on the row axis.

In [14]:
sales.pivot_table(
    index = ["Name", "Date"], values = "Revenue", aggfunc = "sum"
).head(10)

Revenue
Name   Date               
Creed  2020-01-01     4430
       2020-01-02    13214
       2020-01-04     3144
       2020-01-05      938
Dwight 2020-01-01     2639
       2020-01-03    11912
       2020-01-05     7771
Jim    2020-01-01     1864
       2020-01-02     8278
       2020-01-03     4226

Switch the __order of strings in the index list to rearrange the levels__ in the pivot table’s MultiIndex. 

In [15]:
sales.pivot_table(
    index = ["Date", "Name"], values = "Revenue", aggfunc = "sum"
).head(10)

Revenue
Date       Name            
2020-01-01 Creed       4430
           Dwight      2639
           Jim         1864
           Michael     7172
           Oscar       9656
2020-01-02 Creed      13214
           Jim         8278
           Michael     6362
           Oscar       8661
2020-01-03 Dwight     11912

# 2 - Stacking and Unstacking 

Sometimes, we may want to move an index level from one axis to another. 

This change offers a different presentation of the data, and we can decide which view we like better.

## Stack

Here’s a reminder of what sales looks like currently:

In [16]:
sales.head()

Date   Name       Customer  Revenue  Expenses
0 2020-01-01  Oscar  Logistics XYZ     5250       531
1 2020-01-01  Oscar    Money Corp.     4406       661
2 2020-01-02  Oscar     PaperMaven     8661      1401
3 2020-01-03  Oscar    PaperGenius     7075       906
4 2020-01-04  Oscar    Paper Pound     2524      1767

Let’s pivot sales to organize revenue by employee name and date. 

We’ll place __dates on the column axis__ and __names on the row axis__:

In [17]:
by_name_and_date = sales.pivot_table(
    index = "Name", 
    columns = "Date", 
    values = "Revenue", 
    aggfunc = "sum"
)

by_name_and_date.head(2) 

Date    2020-01-01  2020-01-02  2020-01-03  2020-01-04  2020-01-05
Name                                                              
Creed       4430.0     13214.0         NaN      3144.0       938.0
Dwight      2639.0         NaN     11912.0         NaN      7771.0

The __stack method__ moves an index level from the column axis to the row axis. 

The next example moves the Date index level from the column axis to the row axis. 

Pandas creates a __MultiIndex__ to store the two row levels: Name and Date. 

In this example, because only one column of values remains, pandas returns a Series:

In [18]:
by_name_and_date.stack().head(7)

Name    Date      
Creed   2020-01-01     4430.0
        2020-01-02    13214.0
        2020-01-04     3144.0
        2020-01-05      938.0
Dwight  2020-01-01     2639.0
        2020-01-03    11912.0
        2020-01-05     7771.0
dtype: float64

## Unstack

Lets create a pivot table which groups revenue by customer and salesman. 

The __row axis has a two-level MultiIndex__, and the column axis has a regular index:

In [19]:
sales_by_customer = sales.pivot_table(
    index = ["Customer", "Name"],
    values = "Revenue",
    aggfunc = "sum"
)

sales_by_customer.head()

Revenue
Customer          Name            
Average Paper Co. Creed      13214
                  Jim         2287
Best Paper Co.    Dwight      2703
                  Michael    15754
Logistics XYZ     Dwight      9209

The unstack method moves the innermost level of the row index to the column index:

In [20]:
sales_by_customer.unstack()

Revenue                                 
Name                 Creed  Dwight     Jim  Michael   Oscar
Customer                                                   
Average Paper Co.  13214.0     NaN  2287.0      NaN     NaN
Best Paper Co.         NaN  2703.0     NaN  15754.0     NaN
Logistics XYZ          NaN  9209.0     NaN   7172.0  5250.0
Money Corp.         5368.0     NaN  8278.0      NaN  4406.0
Paper Pound            NaN  7771.0  4226.0      NaN  5317.0
PaperGenius            NaN  2639.0  1864.0  12344.0  7075.0
PaperMaven          3144.0     NaN  3868.0      NaN  8661.0

We can specify the level to unstack (by position):

In [21]:
sales_by_customer.unstack(0)

Revenue                                           \
Customer Average Paper Co. Best Paper Co. Logistics XYZ Money Corp.   
Name                                                                  
Creed              13214.0            NaN           NaN      5368.0   
Dwight                 NaN         2703.0        9209.0         NaN   
Jim                 2287.0            NaN           NaN      8278.0   
Michael                NaN        15754.0        7172.0         NaN   
Oscar                  NaN            NaN        5250.0      4406.0   

                                             
Customer Paper Pound PaperGenius PaperMaven  
Name                                         
Creed            NaN         NaN     3144.0  
Dwight        7771.0      2639.0        NaN  
Jim           4226.0      1864.0     3868.0  
Michael          NaN     12344.0        NaN  
Oscar         5317.0      7075.0     8661.0

We can also specify the level to unstack giving the name

In [22]:
sales_by_customer.unstack("Customer")

Revenue                                           \
Customer Average Paper Co. Best Paper Co. Logistics XYZ Money Corp.   
Name                                                                  
Creed              13214.0            NaN           NaN      5368.0   
Dwight                 NaN         2703.0        9209.0         NaN   
Jim                 2287.0            NaN           NaN      8278.0   
Michael                NaN        15754.0        7172.0         NaN   
Oscar                  NaN            NaN        5250.0      4406.0   

                                             
Customer Paper Pound PaperGenius PaperMaven  
Name                                         
Creed            NaN         NaN     3144.0  
Dwight        7771.0      2639.0        NaN  
Jim           4226.0      1864.0     3868.0  
Michael          NaN     12344.0        NaN  
Oscar         5317.0      7075.0     8661.0

# 3 - Melting 

Melting is the process of converting a wide data set to a narrow one.

### Dataset

We will use for the next examples a dataset of video games sales

In [23]:
video_game_sales = pd.read_csv("video_game_sales.csv")
video_game_sales.head()

Name     NA     EU     JP  Other
0                Wii Sports  41.49  29.02   3.77   8.46
1         Super Mario Bros.  29.08   3.58   6.81   0.77
2            Mario Kart Wii  15.85  12.88   3.79   3.31
3         Wii Sports Resort  15.75  11.01   3.28   2.96
4  Pokemon Red/Pokemon Blue  11.27   8.89  10.22   1.00

Each row includes the game’s name as well as the number of units sold (in millions) in the North America (NA), Europe (EU), Japan (JP), and other (Other) regions:

In [24]:
video_game_sales.head(1)

Name     NA     EU    JP  Other
0  Wii Sports  41.49  29.02  3.77   8.46

video_game_sales stores its data in wide format

Four columns (NA, EU, JP, and Other) store the same data point: the number of units sold. 

If we added more regional sales columns, the data set would grow horizontally.


### Reshaping it

With __melt__ method we can move the values "NA", "EU", "JP", and "Other" to a new Region column

The  melt method accepts two primary __parameters__:

* The __id_vars__ parameter sets the identifier column, the column for which the wide data set aggregates data. Name is the identifier column in video_game_sales. The data set aggregates sales per video game.

* The value_vars parameter accepts the column(s) whose values pandas will melt and store in a new column.

In [25]:
video_game_sales.melt(id_vars = 'Name', value_vars = "NA").head()

Name variable  value
0                Wii Sports       NA  41.49
1         Super Mario Bros.       NA  29.08
2            Mario Kart Wii       NA  15.85
3         Wii Sports Resort       NA  15.75
4  Pokemon Red/Pokemon Blue       NA  11.27

In a way, we are unpivoting the video_game_sales DataFrame

To __melt all four of the regional sales columns__ we need to pass the value_vars parameter a list of the four regional sales columns from video_game_sales

In [26]:
regional_sales_columns = ["NA", "EU", "JP", "Other"]

video_game_sales.melt(
    id_vars = "Name", value_vars = regional_sales_columns
)

Name variable  value
0                                            Wii Sports       NA  41.49
1                                     Super Mario Bros.       NA  29.08
2                                        Mario Kart Wii       NA  15.85
3                                     Wii Sports Resort       NA  15.75
4                              Pokemon Red/Pokemon Blue       NA  11.27
...                                                 ...      ...    ...
66259                Woody Woodpecker in Crazy Castle 5    Other   0.00
66260                     Men in Black II: Alien Escape    Other   0.00
66261  SCORE International Baja 1000: The Official Game    Other   0.00
66262                                        Know How 2    Other   0.00
66263                                  Spirits & Spells    Other   0.00

[66264 rows x 3 columns]

The melt method returns a DataFrame with 66,264 rows! 

The __variable__ column holds the four regional column names from video_game_sales. 

The __value__ column holds the values from those four regional sales columns. 


We can customize the melted DataFrame’s column names by passing arguments to the __var_name__ and __value_name__ parameters. 

The next example uses Region for the variable column and Sales for the value column:

In [27]:
video_game_sales_by_region = video_game_sales.melt(
    id_vars = "Name", 
    value_vars = regional_sales_columns,
    var_name = "Region",
    value_name = "Sales"
)

video_game_sales_by_region.head()

Name Region  Sales
0                Wii Sports     NA  41.49
1         Super Mario Bros.     NA  29.08
2            Mario Kart Wii     NA  15.85
3         Wii Sports Resort     NA  15.75
4  Pokemon Red/Pokemon Blue     NA  11.27

### narrow data and pivoting

__Narrow data is easier to aggregate than wide data__. 

Let’s say we want to find the sum of each video game’s sales across all regions. 

Given the melted data set, we can use the pivot_table method to accomplish this task with a few lines of code.

We can even sort by sales and get only the top ten.

In [28]:
video_game_sales_by_region.pivot_table(
    index = "Name", values = "Sales", aggfunc = "sum"
).sort_values("Sales",ascending=False).head(10)

Sales
Name                                 
Wii Sports                      82.74
Grand Theft Auto V              55.92
Super Mario Bros.               45.31
Tetris                          35.84
Mario Kart Wii                  35.83
Wii Sports Resort               33.00
Pokemon Red/Pokemon Blue        31.38
Call of Duty: Black Ops         31.04
Call of Duty: Modern Warfare 3  30.84
New Super Mario Bros.           30.01

Now by Region (combining all games)...

In [29]:
video_game_sales_by_region.pivot_table(
    index = "Region", values = "Sales", aggfunc = "sum"
).head()

Sales
Region         
EU      2434.09
JP      1284.11
NA      4392.64
Other    797.65

# 4 - Exploding a List of Values

Sometimes, a data set stores multiple values in the same cell. 

We may want to break up the data cluster so that each row stores a single value.

### Example

Let's consider recipes.csv, a collection of three recipes, each of which has a name and an ingredients list. 

The ingredients are stored in a single comma-separated string:

In [30]:
recipes = pd.read_csv("recipes.csv")
recipes

Recipe                                        Ingredients
0   Cashew Crusted Chicken  Apricot preserves, Dijon mustard, curry powder...
1      Tomato Basil Salmon  Salmon filets, basil, tomato, olive oil, Parme...
2  Parmesan Cheese Chicken  Bread crumbs, Parmesan cheese, Italian seasoni...

__str.split__ method uses a delimiter to split a string into substrings. 

We can split each Ingredients string by the presence of a comma. 

In the next example, pandas returns a Series of lists. Each list stores the ingredients for the row:

In [31]:
recipes["Ingredients"].str.split(",")

0    [Apricot preserves,  Dijon mustard,  curry pow...
1    [Salmon filets,  basil,  tomato,  olive oil,  ...
2    [Bread crumbs,  Parmesan cheese,  Italian seas...
Name: Ingredients, dtype: object

We can overwrite the original Ingredients column with the new one:

In [32]:
recipes["Ingredients"] = recipes["Ingredients"].str.split(",")
recipes

Recipe                                        Ingredients
0   Cashew Crusted Chicken  [Apricot preserves,  Dijon mustard,  curry pow...
1      Tomato Basil Salmon  [Salmon filets,  basil,  tomato,  olive oil,  ...
2  Parmesan Cheese Chicken  [Bread crumbs,  Parmesan cheese,  Italian seas...

Now, how can we spread out each list’s values across multiple rows? 

The __explode__ method creates a separate row for each list element in a Series. 

We invoke the method on a DataFrame and pass in the column with lists:

In [33]:
recipes.explode("Ingredients")

Recipe         Ingredients
0   Cashew Crusted Chicken   Apricot preserves
0   Cashew Crusted Chicken       Dijon mustard
0   Cashew Crusted Chicken        curry powder
0   Cashew Crusted Chicken     chicken breasts
0   Cashew Crusted Chicken             cashews
1      Tomato Basil Salmon       Salmon filets
1      Tomato Basil Salmon               basil
1      Tomato Basil Salmon              tomato
1      Tomato Basil Salmon           olive oil
1      Tomato Basil Salmon     Parmesan cheese
2  Parmesan Cheese Chicken        Bread crumbs
2  Parmesan Cheese Chicken     Parmesan cheese
2  Parmesan Cheese Chicken   Italian seasoning
2  Parmesan Cheese Chicken                 egg
2  Parmesan Cheese Chicken     chicken breasts

# 5- MultiIndex

A MultiIndex is an index object that holds multiple levels. 

Each level stores a value for the row. 

It is optimal to use a MultiIndex when a combination of values provides the best identifier for a row of data.

####  Extracting one or more rows with loc

In [48]:
sales_by_customer.head()

Revenue
Customer          Name            
Average Paper Co. Creed      13214
                  Jim         2287
Best Paper Co.    Dwight      2703
                  Michael    15754
Logistics XYZ     Dwight      9209

We can use the loc and iloc accessors to pull rows from a MultiIndex DataFrame

We can pass them in a tuple within the square brackets


In [49]:
sales_by_customer.loc[("Best Paper Co.","Dwight")]

If we pass a single label in the square brackets, pandas looks for it in the outermost MultiIndex level

In [50]:
sales_by_customer.loc["Best Paper Co."]

For more see : https://learning.oreilly.com/library/view/pandas-in-action/9781617297434/OEBPS/Text/07.htm#heading_id_3


# 6 - Coding Challenge

### Datasets

We have two data sets for you to play with. 

The used_cars.csv file is a listing of used cars for sale on the classifieds website Craigslist. 

Each row includes the car’s manufacturer, year of production, fuel type, transmission type, and price:

In [34]:
cars = pd.read_csv("used_cars.csv")
cars.head()

Manufacturer  Year Fuel Transmission  Price
0        Acura  2012  Gas    Automatic  10299
1       Jaguar  2011  Gas    Automatic   9500
2        Honda  2004  Gas    Automatic   3995
3    Chevrolet  2016  Gas    Automatic  41988
4          Kia  2015  Gas    Automatic  12995

The minimum_wage.csv data set is a collection of minimum wages across the United States. 

The data set has a State column and multiple year columns:

In [35]:
min_wage = pd.read_csv("minimum_wage.csv")
min_wage.head()

State  2010  2011  2012  2013  2014  2015   2016   2017
0     Alabama  0.00  0.00  0.00  0.00  0.00  0.00   0.00   0.00
1      Alaska  8.90  8.63  8.45  8.33  8.20  9.24  10.17  10.01
2     Arizona  8.33  8.18  8.34  8.38  8.36  8.50   8.40  10.22
3    Arkansas  7.18  6.96  6.82  6.72  6.61  7.92   8.35   8.68
4  California  9.19  8.91  8.72  8.60  9.52  9.51  10.43  10.22

### Challenges

* Aggregate the sum of car prices in cars. Group the results by fuel type on the row axis.

* Aggregate the count of cars in cars. Group the results by manufacturer on the index axis and transmission type on the column axis. Show the subtotals for both the rows and columns.

* Aggregate the average of car prices in cars. Group the results by year and fuel type on the index axis and transmission type on the column axis.

* Given a DataFrame from the preceding challenge, move the transmission level from the column axis to the row axis.

* Convert the min_wage from wide format to narrow format. In other words, move the data from the eight year columns (2010–17) to a single column.

### Possible solutions

##### 1 - Aggregate the sum of car prices in cars. Group the results by fuel type on the row axis.

aggregate the sum  ==> aggfunc = "sum"

aggregate prices     ==> values = "Price"

group by fuel type on row axis   ==> index =  "Fuel"

In [36]:
cars.pivot_table(values="Price",
                 index = "Fuel",
                 aggfunc= "sum"
                )

Price
Fuel                 
Diesel      986177143
Electric     18502957
Gas       86203853926
Hybrid       44926064
Other       242096286

##### 2 - Aggregate the count of cars in cars. 

##### Group the results by manufacturer on the index axis and transmission type on the column axis. 

##### Show the subtotals for both the rows and columns.

In [37]:
cars.head(2)

Manufacturer  Year Fuel Transmission  Price
0        Acura  2012  Gas    Automatic  10299
1       Jaguar  2011  Gas    Automatic   9500

aggregate the count ==> aggfunc = "count"

results by manufacturer  ==> index = "Manufacturer"

transmission type on columns ==> columns = "Transmission"

show subtotals ==> margins = True

In [38]:
cars.pivot_table(values="Price",
                 index = "Manufacturer",
                 columns = "Transmission",
                 aggfunc= "count",
                 margins = True
                ).tail()

Transmission  Automatic   Manual    Other     All
Manufacturer                                     
Tesla             179.0      NaN     59.0     238
Toyota          31480.0   1367.0   2134.0   34981
Volkswagen       7985.0   1286.0    236.0    9507
Volvo            2665.0    155.0     50.0    2870
All            398428.0  21005.0  21738.0  441171

##### 3 - Aggregate the average of car prices in cars. 

##### Group the results by year and fuel type on the index axis and transmission type on the column axis.



aggregate the average ==> aggfunc = "mean"

results by year AND fuel  ==> index = ["Year","Fuel"]

transmission type on columns ==> columns = "Transmission"


In [39]:
cars.pivot_table(values="Price",
                 index = ["Year","Fuel"],
                 columns = "Transmission",
                 aggfunc= "mean",
                )

Transmission      Automatic        Manual         Other
Year Fuel                                              
2000 Diesel    11326.176962  14010.164021  11075.000000
     Electric   1500.000000           NaN           NaN
     Gas        4314.675996   6226.140327   3203.538462
     Hybrid     2600.000000   2400.000000           NaN
     Other     16014.918919  11361.952381  12984.642857
...                     ...           ...           ...
2020 Diesel    63272.595930      1.000000   1234.000000
     Electric   8015.166667   2200.000000  20247.500000
     Gas       34925.857933  36007.270833  20971.045455
     Hybrid    35753.200000           NaN   1234.000000
     Other     22210.306452           NaN   2725.925926

[102 rows x 3 columns]

##### 4 - Given a DataFrame from the preceding challenge, move the transmission level from the column axis to the row axis.

In [40]:
result = cars.pivot_table(values="Price",
                 index = ["Year","Fuel"],
                 columns = "Transmission",
                 aggfunc= "mean",
                )

To move columns to the indes we can use __stack()__

In [41]:
result.stack()

Year  Fuel      Transmission
2000  Diesel    Automatic       11326.176962
                Manual          14010.164021
                Other           11075.000000
      Electric  Automatic        1500.000000
      Gas       Automatic        4314.675996
                                    ...     
2020  Gas       Other           20971.045455
      Hybrid    Automatic       35753.200000
                Other            1234.000000
      Other     Automatic       22210.306452
                Other            2725.925926
Length: 274, dtype: float64

##### 5 - Convert the min_wage from wide format to narrow format. 

##### In other words, move the data from the eight year columns (2010–17) to a single column.

In [42]:
min_wage.head(2)

State  2010  2011  2012  2013  2014  2015   2016   2017
0  Alabama   0.0  0.00  0.00  0.00   0.0  0.00   0.00   0.00
1   Alaska   8.9  8.63  8.45  8.33   8.2  9.24  10.17  10.01

The solution is the __melt__ method. 

We need to identify __id_vars__ and __value_vars__...

We can declare the State column as the __identifier column__ (id_vars) and the eight year columns as the variable columns (value_vars):

In [43]:
year_columns = min_wage.columns[1:]
print(year_columns)
min_wage.melt(id_vars = "State", value_vars = year_columns)

Index(['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017'], dtype='object')


State variable  value
0          Alabama     2010   0.00
1           Alaska     2010   8.90
2          Arizona     2010   8.33
3         Arkansas     2010   7.18
4       California     2010   9.19
..             ...      ...    ...
435       Virginia     2017   7.41
436     Washington     2017  11.24
437  West Virginia     2017   8.94
438      Wisconsin     2017   7.41
439        Wyoming     2017   5.26

[440 rows x 3 columns]

We can also customize the column names with the __var_name__ and __value_name__ parameters.

In [44]:
min_wage.melt(id_vars = "State", 
              value_vars = year_columns,
              var_name = "Year",
              value_name = "Wage"
             )

State  Year   Wage
0          Alabama  2010   0.00
1           Alaska  2010   8.90
2          Arizona  2010   8.33
3         Arkansas  2010   7.18
4       California  2010   9.19
..             ...   ...    ...
435       Virginia  2017   7.41
436     Washington  2017  11.24
437  West Virginia  2017   8.94
438      Wisconsin  2017   7.41
439        Wyoming  2017   5.26

[440 rows x 3 columns]